In [110]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
import seaborn as sns

In [75]:
data = pd.read_csv('QueryResults_2.csv')

In [76]:
data

Title  \
0                          pandas equivalent of np.where   
1      The type system in Scala is Turing complete. P...   
2      How can you publish a ClickOnce application th...   
3               How do you get a directory listing in C?   
4      Expression Evaluation and Tree Walking using p...   
...                                                  ...   
49995        Frontend javascript frameworks with node.js   
49996  WELD-001408: Unsatisfied dependencies for type...   
49997           Optimize uploaded images with php (jpeg)   
49998  Google Maps v2 - set both my location and zoom in   
49999  post request with multiple parameters JSON and...   

                                                    Body  \
0      <p><code>np.where</code> has the semantics of ...   
1      <p>There are claims that Scala's type system i...   
2      <p>I have CruiseControl.NET Version 1.4 set up...   
3      <p>How do you scan a directory for folders and...   
4      <p>This morning, I was reading <a href="http:/...   
...                                                  ...   
49995  <p>I am starting to learn frontend JavaScript ...   
49996  <p>I'm a Java EE-newbie. I want to test JSF an...   
49997  <p>When running Page Speed in Google Chrome it...   
49998  <p>My question is, does anyone know how to set...   
49999  <p>I've created a rest api using Jersey/Jackso...   

                                                    Tags        Id  Score  \
0                  <python><pandas><numpy><where-clause>  38579532     70   
1      <language-agnostic><scala><type-systems><turin...   4047512     57   
2       <msbuild><clickonce><cruisecontrol.net><publish>     12482     24   
3      <c><file><directory><cross-platform><common-ta...     12489     65   
4            <oop><recursion><polymorphism><binary-tree>     12516     27   
...                                                  ...       ...    ...   
49995  <javascript><node.js><backbone.js><ember.js><j...  14072131     30   
49996  <jsf><jakarta-ee><dependency-injection><cdi><m...  28352461     53   
49997  <php><gd><jpeg><image-compression><image-optim...  14072456     16   
49998     <android><api><google-maps><google-maps-api-2>  14074129    125   
49999                <java><json><post><jersey><jackson>  14075422     22   

       ViewCount  FavoriteCount  AnswerCount  
0         106887             17            1  
1           7625             12            2  
2          12992             13            5  
3         135968             21            9  
4           7032             19           16  
...          ...            ...          ...  
49995      29357             11            2  
49996     135308             12            7  
49997      40714             12            4  
49998     163166             41           11  
49999      71183             19            2  

[50000 rows x 8 columns]

### Les titres sont tout aussi important que le Body. Nous pouvons regrouper les deux. 

In [77]:
data.Body = data.Title + " " + data.Body 

In [78]:
data.Body

0        pandas equivalent of np.where <p><code>np.wher...
1        The type system in Scala is Turing complete. P...
2        How can you publish a ClickOnce application th...
3        How do you get a directory listing in C? <p>Ho...
4        Expression Evaluation and Tree Walking using p...
                               ...                        
49995    Frontend javascript frameworks with node.js <p...
49996    WELD-001408: Unsatisfied dependencies for type...
49997    Optimize uploaded images with php (jpeg) <p>Wh...
49998    Google Maps v2 - set both my location and zoom...
49999    post request with multiple parameters JSON and...
Name: Body, Length: 50000, dtype: object

## Gestion des TAGS ( cible ) 

#### Définition des fonction permettant de cleaner les TAGS

In [79]:
def tag_remover(tag):
    tag = tag.replace('>' , " ").replace('<', " ")
    return tag

def tag_to_word_list(tag):
    tag = BeautifulSoup(tag).get_text()
    tag = tag.split()
    return(tag)

#### Parcourons tous les TAGS pour leur appliquer nos fonctions de clean.

In [80]:
Tags = []
for tag in data.Tags:
    tag = tag_remover(tag)
    tag = tag_to_word_list(tag)
    Tags.append(tag)
    
tag_bank = [] 
for tag in Tags : 
    for y in tag:
        tag_bank.append(y)

#### Identifions, les TAGS les plus courants, cela pourra certainement nous être utile.

In [81]:
sns.set(font_scale = 1)

In [82]:
import matplotlib.pylab as plt
tags = pd.DataFrame()
tags['Tags'] = tag_bank
# for legend text
plt.figure(figsize = (10,5))

sns.barplot(tags['Tags'].value_counts().head(10).index,\
            tags['Tags'].value_counts().head(10))

top_ten = tags['Tags'].value_counts().head(10).index
top_ten_tags=[]
for i in range(len(top_ten)):
    top_ten_tags.append(top_ten[i])

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [83]:
tags['Tags'].value_counts().head(10).sum()

37339

In [84]:
#Création des data que nous utiliserons pour créer notre modèle supervisé
data_to_train = [] #--> X
target_to_train = [] #--> y
counter = 0
for tag in Tags:
    for i in tag:
        if i in top_ten_tags:            
            data_to_train.append(data.Body[counter])
            target_to_train.append(i)
            
            
    counter+=1

In [85]:
for tag in top_ten:
    print(tag)
    top_ten_tags.append(tag)

java
python
c#
javascript
android
ios
c++
.net
html
iphone


# Préparons nos données à l'analyse textuelle

### Fonctions primaires 

In [86]:

def lemma_fct(list_words) :
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

### Fonction déstinée à la création d'un "Bag of Word"

In [87]:
def question_to_words_BW(question):
    # On récupère le texte
    question_text = BeautifulSoup(question).get_text()
    
    #On ne garde que les caractères intéressants. 
    question = re.sub("[^a-zA-Z+#]"," ", question_text)
    
    # On tokenize nos questions en mots.
    words = question.lower().split()
    
    # On retire les mots clés plus courants avec peu de valeur ajoutée. 
    stops = set(stopwords.words('english'))
    meaningful_words = [w for w in words if not w in stops]
        
    # On lemmatize les mots
    meaningful_words = lemma_fct(meaningful_words)
    
    #On retourne une liste comprenant les mots ainsi récupérés. 
    return(" ".join( meaningful_words))

### Fonctions déstinée à la création du modèle Word2vec. 

In [88]:
def sentence_to_wordlist(sentence, remove_stopwords=False, remove_ponctuation=False,lemmatize = False):

    # 1. Remove HTML
    sentence = BeautifulSoup(sentence).get_text()
    sentence = re.sub("[^a-zA-Z+#]"," ", sentence)
    # 1. Tokenization des phrases en liste de mots. 
    words = sentence.split()    
    # 2. Lemmatisation des listes de mots(en option)
    if lemmatize:
        words = lemma_fct(words)
    
    # 3. Gestion des stopWords (en option)
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        
    # 4. Gestion de la ponctuation (en option)
    if remove_ponctuation:
        ponct = ['[', ']', ',', '.', ':', '?', '(', ')','\'','\"','`']
        words = [w for w in words if not w in ponct]
    
    # 5. Return a list of words
    return(words)

In [89]:
def question_to_sentences(question, tokenizer):
    # Function to split a question into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    
    question = BeautifulSoup(question).get_text()
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(question.strip())
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(sentence_to_wordlist(raw_sentence, remove_stopwords=False,\
                                                  remove_ponctuation=False, lemmatize=False))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [90]:
#tokenizer qui décompose en phrases via la ponctuation= punkt
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# BAG Of WORDS

### Préprocessing

In [91]:
questions_cleared = []
for question in data.Body:
        cleared_question = question_to_words_BW(question)
        questions_cleared.append(cleared_question)
        
# On créé le vectorizer qui va vectorizer les mots de nos questions. 
#vectorizer = CountVectorizer(analyzer = "word", max_features = 5000) 
vectorizer = TfidfVectorizer(analyzer = "word", max_features = 5000)


# On fit notre vectorizer avec nos questions. 
questions_features = vectorizer.fit_transform(questions_cleared)
#questions_features = questions_features.toarray()

### Modèle de prédiction

In [92]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20

lda = LatentDirichletAllocation(n_components = n_topics, max_iter = 5, learning_method='online',learning_offset=50.,random_state=0)

lda.fit(questions_features)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, n_components=20, random_state=0)

#### Ici, notre modèle nous renvoie un vecteur de la taille n_topics, avec pour chacun d'entre eux la probabilité d'appartencance de notre quetioni en input 

In [93]:
lda.transform(questions_features[1])

array([[0.01065472, 0.01065472, 0.01065472, 0.01065472, 0.01065472,
        0.01065472, 0.01065472, 0.01065472, 0.01065472, 0.01065472,
        0.79756031, 0.01065472, 0.01065472, 0.01065472, 0.01065472,
        0.01065472, 0.01065472, 0.01065472, 0.01065472, 0.01065472]])

### Affichage des topics trouvés par notre modèle.


In [94]:
def display_topics(model, feature_names, no_top_words):
    topic_list = pd.DataFrame()
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        topic_list[topic_idx] = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

no_top_words = 5
display_topics(lda,vectorizer.get_feature_names(), 5)

Topic 0:
file project command build python
Topic 1:
keyboard vim modal cursor shortcut
Topic 2:
git branch commit repository github
Topic 3:
map latitude longitude coordinate lat
Topic 4:
self swift nsstring nil view
Topic 5:
curl ch solr bla addr
Topic 6:
df dataframe numpy panda np
Topic 7:
firstname employee mongoose lastname xpath
Topic 8:
tooltip bbb ccc fff brand
Topic 9:
user http request file app
Topic 10:
class value function object like
Topic 11:
datatable insecure treeview tbl parentid
Topic 12:
string character date convert utf
Topic 13:
database sql spring table mysql
Topic 14:
urllib na fp executor executorservice
Topic 15:
node npm module typescript mutex
Topic 16:
emacs lisp myfunction trans theme
Topic 17:
tensorflow gdb tf layer pool
Topic 18:
fixture rax mpi erb rdx
Topic 19:
image android div view color


/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [95]:
from __future__ import print_function
import pyLDAvis

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, questions_features, vectorizer)

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp mod

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.108312 -0.138267       1        1  37.236452
9      0.161088 -0.145453       2        1  24.048938
0      0.198758 -0.128786       3        1  11.979815
19     0.230647 -0.084159       4        1   7.680899
13     0.202992  0.134794       5        1   2.448561
12     0.218799  0.106115       6        1   2.005062
4      0.026388  0.221702       7        1   1.549259
2      0.007477  0.190672       8        1   1.440863
6     -0.053815  0.225017       9        1   1.270761
1     -0.072480  0.039464      10        1   1.141011
15    -0.110210  0.008762      11        1   1.025540
3     -0.117662  0.012827      12        1   1.025271
17    -0.118738 -0.021901      13        1   0.956986
7     -0.116133 -0.042131      14        1   0.935829
14    -0.105674 -0.055535      15        1   0.898516
5     -0.102464 -0.059862      16        1   0.887756
16    -0.091523 -0.065205      17        1   0.873223
11    -0.091643 -0.065178      18        1   0.870528
18    -0.088079 -0.066193      19        1   0.863834
8     -0.086040 -0.066683      20        1   0.860896, topic_info=           Term        Freq       Total Category  logprob  loglift
4245     string  866.000000  866.000000  Default  30.0000  30.0000
2612        map  490.000000  490.000000  Default  29.0000  29.0000
2901       node  469.000000  469.000000  Default  28.0000  28.0000
1850        git  442.000000  442.000000  Default  27.0000  27.0000
3917       self  412.000000  412.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
332   available    0.361343  131.430007  Topic20  -8.6538  -1.1414
2511        lnk    0.361343    9.260961  Topic20  -8.6538   1.5112
957         crt    0.361343   14.002478  Topic20  -8.6538   1.0978
2786     msvcrt    0.361343    6.618136  Topic20  -8.6538   1.8472
2845        nav    0.361343   30.374326  Topic20  -8.6538   0.3234

[1046 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
2         2  0.624047                  aac
3        10  0.866399                  aar
6         3  0.613365                  abi
14        5  0.374143  abstractbeanfactory
47        3  0.357989           actionpack
...     ...       ...                  ...
4984     19  0.898330                  ymm
4988      2  0.027198                 yyyy
4988      6  0.897518                 yyyy
4996      4  0.912311                 zoom
4998      8  0.834049                  zsh

[1865 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 10, 1, 20, 14, 13, 5, 3, 7, 2, 16, 4, 18, 8, 15, 6, 17, 12, 19, 9])

### Création de la Pipeline regroupant le préprocessing et le modèle LDA

In [96]:
# Pour chaque question, on applique la fonction qui récupère les mots clés des questions posées. 
def clearing_questions(Body):
    # On créé une liste vide déstinée à receullir les mots de nos questions. 
    questions_cleared =[]
    for question in Body:
        cleared_question = question_to_words_BW(question)
        questions_cleared.append(cleared_question)
        
    return questions_cleared


In [97]:
from sklearn import datasets, preprocessing, model_selection, ensemble, pipeline
from sklearn.experimental import enable_hist_gradient_boosting

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [98]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20

from sklearn.pipeline import Pipeline
transformer_1 = FunctionTransformer(clearing_questions)
"""
pipeline_BOW = Pipeline([
   ('transformer', transformer_1), ('vectorizer',CountVectorizer(analyzer = "word", max_features = 5000)),
   ('LDA',LatentDirichletAllocation(n_components = n_topics, max_iter = 5,\
                                                                     learning_method='online',learning_offset=50.,random_state=0))
])
"""
pipeline_BOW = Pipeline([
   ('transformer', transformer_1), ('vectorizer',TfidfVectorizer(analyzer = "word", max_features = 5000)),
   ('LDA',LatentDirichletAllocation(n_components = n_topics, max_iter = 5,\
                                                                     learning_method='online',learning_offset=50.,random_state=0))
])

In [99]:
pipeline_BOW.fit(data.Body)

Pipeline(steps=[('transformer',
                 FunctionTransformer(func=<function clearing_questions at 0x7fc531fad670>)),
                ('vectorizer', TfidfVectorizer(max_features=5000)),
                ('LDA',
                 LatentDirichletAllocation(learning_method='online',
                                           learning_offset=50.0, max_iter=5,
                                           n_components=20, random_state=0))])

In [100]:
def create_topic_list(model, feature_names, no_top_words):
    topic_list = pd.DataFrame()
    for topic_idx, topic in enumerate(model.components_):
        topic_list[topic_idx] = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
    return topic_list

def tag_generator(questions):
    for question in questions:
        new_document_topic = np.where(question == question.max())
        new_document_topic = int(new_document_topic[0])
        print('La répartition des proba d\'appartenance aux différents TOPICS est \n  ', question)
        print('Les tags suggérés pour ce document sont :  \n',topic_list.T.iloc[new_document_topic,:5])
    return topic_list.T.iloc[new_document_topic,:5].tolist()



In [101]:
no_top_words = 5
topic_list = create_topic_list(pipeline_BOW['LDA'],pipeline_BOW['vectorizer'].get_feature_names_out(),no_top_words)

### Essai de la pipeline avant API

In [102]:
test_text = pipeline_BOW.transform([data.Body[10]])
tag_generator(test_text)

La répartition des proba d'appartenance aux différents TOPICS est 
   [0.21675889 0.00917852 0.00917852 0.00917852 0.00917852 0.00917852
 0.00917852 0.00917852 0.00917852 0.00917852 0.61802769 0.00917852
 0.00917852 0.00917852 0.00917852 0.00917852 0.00917852 0.00917852
 0.00917852 0.00917852]
Les tags suggérés pour ce document sont :  
 0       class
1       value
2    function
3      object
4        like
Name: 10, dtype: object


['class', 'value', 'function', 'object', 'like']

# API Tf-Idf

In [ ]:
from fast_dash import FastDash
from fast_dash.Components import Text

# Step 1: Define your model inference
def text_to_text_function(question):
    
    result = pipeline_BOW.transform([question])
    result = tag_generator(result)
    
    return result

# Step 2: Specify the input and output components
app = FastDash(callback_fn=text_to_text_function, 
                inputs=Text, 
                outputs=Text, 
                title='Unsupervised Question TAG Generator')

# Step 3: Run your app!
app.run()

# * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)

# WORD2VEC Model

In [103]:
sentences = []
for question in data['Body']:
    sentences += (question_to_sentences(question,tokenizer))

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://stackoverflow.com/questions/66268/what-is-the-best-cc-coding-style-closed" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://lists.webkit.org/pipermail/webkit-dev/2008-September/005019.html
https://lists.webkit.org/pipermail/webkit-dev/2009-June/008182.html" looks like a URL. Be

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.uploadify.com/demos/
http://www.zamzar.com/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://cran.r-project.org/web/packages/gcbd/vignettes/gcbd.pdf." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://jsfiddle.net/RagKH/" looks like a URL. Beautiful Soup is not an HTTP c

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.outlinenone.com/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://github.com/MarkNijhof/Fohjin
http://cre8ivethought.com/blog/2009/11/12/cqrs--la-greg-young/
http://cre8ivethought.com/blog/2009/11/28/cqrs-trying-to-make-it-re-usable/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarn

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://github.com/parkwookyun/firebase-angular-board" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://en.cppreference.com/w/cpp/atomic/memory_order" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://jsfiddle.net/4wqEm/2/" looks like a URL. Beautiful Soup is not an HTTP client. You 

In [104]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = 5      # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)



/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
2022-04-20 20:04:39,571 : INFO : collecting all words and their counts
2022-04-20 20:04:39,571 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-04-20 20:04:39,596 : INFO : PROGRESS: at sentence #10000, processed 202430 words, keeping 15018 word types
2022-04-20 20:04:39,624 : INFO : PROGRESS: at sentence #20000, processed 420872 words, keeping 23939 word types
2022-04-20 20:04:39,651 : INFO : PROGRESS: at sentence #30000, processed 635795 words, keeping 30743 word types
2022-04-20 20:04:39,679 : INFO : PROGRESS: at sentence #40000, processed 863534 words, keeping 37826 word types
2022-04-20 20:04:39,705 : INFO : PROGRESS: at sentence #50000, processed 1069872 words, keeping 42

Training model...


2022-04-20 20:04:39,822 : INFO : PROGRESS: at sentence #90000, processed 1993026 words, keeping 64598 word types
2022-04-20 20:04:39,852 : INFO : PROGRESS: at sentence #100000, processed 2225962 words, keeping 69693 word types
2022-04-20 20:04:39,883 : INFO : PROGRESS: at sentence #110000, processed 2477671 words, keeping 75076 word types
2022-04-20 20:04:39,914 : INFO : PROGRESS: at sentence #120000, processed 2719092 words, keeping 80406 word types
2022-04-20 20:04:39,945 : INFO : PROGRESS: at sentence #130000, processed 2962169 words, keeping 85446 word types
2022-04-20 20:04:39,978 : INFO : PROGRESS: at sentence #140000, processed 3194443 words, keeping 89723 word types
2022-04-20 20:04:40,010 : INFO : PROGRESS: at sentence #150000, processed 3408755 words, keeping 93329 word types
2022-04-20 20:04:40,041 : INFO : PROGRESS: at sentence #160000, processed 3627376 words, keeping 96898 word types
2022-04-20 20:04:40,072 : INFO : PROGRESS: at sentence #170000, processed 3855928 words, 

2022-04-20 20:04:54,099 : INFO : EPOCH 4 - PROGRESS: at 26.20% examples, 1548097 words/s, in_qsize 9, out_qsize 0
2022-04-20 20:04:55,105 : INFO : EPOCH 4 - PROGRESS: at 51.34% examples, 1568850 words/s, in_qsize 9, out_qsize 0
2022-04-20 20:04:56,108 : INFO : EPOCH 4 - PROGRESS: at 76.24% examples, 1558888 words/s, in_qsize 9, out_qsize 0
2022-04-20 20:04:57,071 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-04-20 20:04:57,076 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-04-20 20:04:57,079 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-04-20 20:04:57,080 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-20 20:04:57,082 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-20 20:04:57,083 : INFO : EPOCH - 4 : training on 7922847 raw words (6262912 effective words) took 4.0s, 1567660 effective words/s
2022-04-20 20:04:58,092 : INFO : EPOCH 5 - PROGRESS: at 25.94% ex

## Embedding du modèle Word2Vec

In [105]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index_to_key)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model.wv[word])
    # 
    
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")

    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [106]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_questions = []
# Ici, data_to_train a été défini dans la partie "Gestion des Tags". Cela correspond aux données avec 
# au moins 1 des 10 tags les plus récurents. C'est sur ces données que l'on va entrainer notre modèle supervisé. 
for question in data_to_train:
    clean_train_questions.append(sentence_to_wordlist( question, \
        remove_stopwords=True ))
    
print ("Creating average feature vecs for test questions")
trainDataVecs = getAvgFeatureVecs(clean_train_questions, model, num_features)

Creating average feature vecs for test questions
Review 0 of 37339
Review 1000 of 37339
Review 2000 of 37339
Review 3000 of 37339
Review 4000 of 37339
Review 5000 of 37339
Review 6000 of 37339
Review 7000 of 37339
Review 8000 of 37339
Review 9000 of 37339
Review 10000 of 37339
Review 11000 of 37339
Review 12000 of 37339
Review 13000 of 37339
Review 14000 of 37339
Review 15000 of 37339
Review 16000 of 37339
Review 17000 of 37339
Review 18000 of 37339
Review 19000 of 37339
Review 20000 of 37339
Review 21000 of 37339
Review 22000 of 37339
Review 23000 of 37339
Review 24000 of 37339
Review 25000 of 37339
Review 26000 of 37339
Review 27000 of 37339
Review 28000 of 37339
Review 29000 of 37339
Review 30000 of 37339
Review 31000 of 37339
Review 32000 of 37339
Review 33000 of 37339
Review 34000 of 37339
Review 35000 of 37339
Review 36000 of 37339
Review 37000 of 37339


In [114]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
target_to_train_encoded = label_encoder.fit_transform(target_to_train)

X_train, X_test, y_train, y_test = train_test_split(trainDataVecs,target_to_train_encoded,test_size=0.3)

In [115]:
from sklearn.datasets import load_iris
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


model_to_set = OneVsRestClassifier(SVC(kernel="poly"))

parameters = {
    "estimator__C": [0.1,1,10],
    "estimator__kernel": ["poly","rbf"],
    "estimator__degree":[1, 2, 3],
}

model_tunning = GridSearchCV(model_to_set, param_grid=parameters)

model_tunning.fit(X_train[0:1000], y_train[0:1000])

print (model_tunning.best_score_)
print (model_tunning.best_params_)


0.664
{'estimator__C': 1, 'estimator__degree': 1, 'estimator__kernel': 'poly'}


In [116]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

#Avec Probas :
clf = OneVsRestClassifier(SVC(probability=True))
clf.fit(X_train[0:1000], y_train[0:1000])

#Sans Probas : 
#clf = OneVsRestClassifier(SVC(probability=False)).fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(probability=True))

### Pipeline

### Créons une pipeline afin de faciliter la prédiction de notre notre modèle. 

In [117]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
target_to_train_encoded = label_encoder.fit_transform(target_to_train)

X_train, X_test, y_train, y_test = train_test_split(data_to_train,target_to_train_encoded,test_size=0.3)

In [118]:
len(target_to_train_encoded)

37339

In [119]:
def cleaning_question_W2V(data_to_train):
    
    clean_train_questions = []
    # Ici, data_to_train a été défini dans la partie "Gestion des Tags". Cela correspond aux données avec 
    # au moins 1 des 10 tags les plus récurents. C'est sur ces données que l'on va entrainer notre modèle supervisé. 
    for question in data_to_train:
        clean_train_questions.append(sentence_to_wordlist( question, \
            remove_stopwords=True ))
    
    print ("Creating average feature vecs for test questions")
    trainDataVecs = getAvgFeatureVecs(clean_train_questions, model, num_features)
    return trainDataVecs

# Création d'un transformeur, permettant d'intégrer notre fonction dans la pipeline. 

from sklearn.preprocessing import FunctionTransformer
transformer_2 = FunctionTransformer(cleaning_question_W2V)

In [120]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

### Création de la pipeline 

In [121]:
pipeline = Pipeline([
   ('transformer', transformer_2), ('clf',OneVsRestClassifier(SVC(probability=True))), 
])

In [122]:
pipeline.fit(X_train, y_train)

Creating average feature vecs for test questions
Review 0 of 26137
Review 1000 of 26137
Review 2000 of 26137
Review 3000 of 26137
Review 4000 of 26137
Review 5000 of 26137
Review 6000 of 26137
Review 7000 of 26137
Review 8000 of 26137
Review 9000 of 26137
Review 10000 of 26137
Review 11000 of 26137
Review 12000 of 26137
Review 13000 of 26137
Review 14000 of 26137
Review 15000 of 26137
Review 16000 of 26137
Review 17000 of 26137
Review 18000 of 26137
Review 19000 of 26137
Review 20000 of 26137
Review 21000 of 26137
Review 22000 of 26137
Review 23000 of 26137
Review 24000 of 26137
Review 25000 of 26137
Review 26000 of 26137


Pipeline(steps=[('transformer',
                 FunctionTransformer(func=<function cleaning_question_W2V at 0x7fc497ee71f0>)),
                ('clf', OneVsRestClassifier(estimator=SVC(probability=True)))])

### Prédiction du classifier

In [123]:
clean_test_questions = []
data_to_test = data.Body[12356:12359]
for question in data_to_test:
    clean_test_questions.append(sentence_to_wordlist( question, \
        remove_stopwords=True ))
    
print ("Creating average feature vecs for test questions")
testDataVecs = getAvgFeatureVecs(clean_test_questions, model, num_features)

prediction = clf.predict(testDataVecs)
prediction = label_encoder.inverse_transform(prediction)
proba = clf.predict_proba(testDataVecs)
for i in range(len(prediction)):
    print('le texte n°',i,'est : \n', BeautifulSoup(data_to_test.iloc[i]).get_text() )
    print('LA PREDICTION EST :' , prediction[i])
    print('VECTEUR DE PROBA', proba[i])

Creating average feature vecs for test questions
Review 0 of 3
le texte n° 0 est : 
 Convert a string representation of a hex dump to a byte array using Java? I am looking for a way to convert a long string (from a dump), that represents hex values into a byte array.
I couldn't have phrased it better than the person that posted the same question here.
But to keep it original, I'll phrase it my own way: suppose I have a string "00A0BF" that I would like interpreted as the
byte[] {0x00,0xA0,0xBf}

what should I do?
I am a Java novice and ended up using BigInteger and watching out for leading hex zeros. But I think it is ugly and I am sure I am missing something simple. 

LA PREDICTION EST : java
VECTEUR DE PROBA [0.06951857 0.00501554 0.14602609 0.03121014 0.03690474 0.02148504
 0.07733405 0.56292812 0.01578561 0.0337921 ]
le texte n° 1 est : 
 Where and When to get data for Watch Complication After working with complications for a few days, I feel confident saying the following about th

### Optional : Clustering

In [ ]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

### Pistes de travail

In [ ]:
probas = pipeline.predict_proba(X_train[200:201])[0]
print(probas)

result = []
for idx, prob in enumerate(probas) : 
    if prob > 0.1:
        result.append(idx)
result = label_encoder.inverse_transform(result)
print(result)


In [ ]:
pipeline.predict(X_train[200:201])[-1]

In [ ]:
probas = pipeline.predict_proba(X_train[200:201])[0]
print(probas)

result = []
for idx, prob in enumerate(probas) : 
    if prob > 0.1:
        result.append(idx)
result = label_encoder.inverse_transform(result)
print(result)


In [ ]:
result_2 =[]
for idx,i in enumerate(result) : 
    result_2.append(i)
    result_2.append(probas[idx])
result_2

# API Word2Vec

In [ ]:
from fast_dash import FastDash
from fast_dash.Components import Text

# Step 1: Define your model inference
def text_to_text_function(question):
    result = []
    result_2  = []
    probas = []
    #result = pipeline.predict([question])
    #result = label_encoder.inverse_transform(result)[0]
    result_proba = pipeline.predict_proba([question])
    print(result_proba)
    for idx, prob in enumerate(result_proba[0]) : 
        if prob > 0.1:
            result.append(idx)
            probas.append(probas)
    print(result)
    result = label_encoder.inverse_transform(result)
    return result

# Step 2: Specify the input and output components
app = FastDash(callback_fn=text_to_text_function, 
                inputs=Text, 
                outputs=Text, 
                title='App title')

# Step 3: Run your app!
app.run()

# * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)